In [1]:
from dolfin import *

import cell_geom as ce
import cell_material as ma
import cell_computation as com

In [2]:
# Set geometry
mesh = Mesh(r"m.xml")
cell = ce.unit_cell(mesh)
inc = ce.Inclusion_Circle()
inc = [inc]
cell.inclusion(inc)

In [3]:
VFS = VectorFunctionSpace(cell.mesh, "CG", 1,
                        constrained_domain=ce.PeriodicBoundary_no_corner())

In [4]:
# Set materials
E_m, nu_m, Kappa_m = 2e5, 0.4, 7.
n = 1000
E_i, nu_i, Kappa_i = 1000*E_m, 0.3, n*Kappa_m

mat_m = ma.neo_hook_mre(E_m, nu_m, Kappa_m)
mat_i = ma.neo_hook_mre(E_i, nu_i, Kappa_i)
mat_li = [mat_m, mat_i]

In [5]:
# Macro Field Boundary
F_bar = [0.9, 0., 0., 1.]
E_bar = [0., 0.]

In [6]:
# Solution Field
w = Function(VFS)
E = Function(VFS)
strain_space_w = TensorFunctionSpace(mesh, 'DG', 0)
strain_space_E = VectorFunctionSpace(mesh, 'CG', 1)
def deform_grad_with_macro(F_bar, w_component):
    return F_bar + grad(w_component)
def e_field_with_macro(E_bar, E):
    return E_bar + E

In [7]:
# Computation Initialization
comp = com.MicroComputation(cell, mat_li, 
                            [deform_grad_with_macro, e_field_with_macro],
                            [strain_space_w, strain_space_E], 1)

comp.input([F_bar, E_bar], [w, E])
comp.comp_fluctuation()

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics@fenicsproject.org
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to solve nonlinear system with NewtonSolver.
*** Reason:  Newton solver did not converge because maximum number of iterations reached.
*** Where:   This error was encountered inside NewtonSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 1.6.0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [ ]:
# Post-Processing
# comp._energy_update()
# comp.comp_strain()
# comp.comp_stress()
# comp.avg_merge_strain()
# comp.avg_merge_stress()
comp.effective_moduli_2()